# Projeto 3: Classificação binária brest cancer com tuning dos parâmetros

## Etapa 1: Importação das bibliotecas

In [1]:
!pip install skorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.7/193.7 KB 2.1 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import sklearn
import skorch
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import GridSearchCV
from skorch import NeuralNetBinaryClassifier

In [3]:
torch.__version__, skorch.__version__, sklearn.__version__

('1.13.1+cu116', '0.12.1', '1.2.1')

## Etapa 2: Base de dados

In [4]:
np.random.seed(123)
torch.manual_seed(123)

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
previsores = pd.read_csv('/content/drive/MyDrive/Deep Learing de A à Z com PyTorch/Bases/entradas_breast.csv')
classe = pd.read_csv('/content/drive/MyDrive/Deep Learing de A à Z com PyTorch/Bases/saidas_breast.csv')

In [7]:
previsores = np.array(previsores, dtype = 'float32')
classe = np.array(classe, dtype = 'float32').squeeze(1)

In [8]:
previsores.shape

(569, 30)

In [9]:
classe.shape

(569,)

## Etapa 3: Classe para estrutura da rede neural

Na versão atual do Skorch, os resultados da rede neural devem ser retornados sem ativação, ou seja, sem a camada sigmoide no final. Com isto, a função de custo deve ser `BCEWithLogitsLoss`.

In [10]:
class classificador_torch(nn.Module):
  def __init__(self, activation, neurons, initializer):
    super().__init__()
    # 30 -> 16 -> 16 -> 1
    self.dense0 = nn.Linear(30, neurons)
    initializer(self.dense0.weight)
    self.activation0 = activation
    self.dense1 = nn.Linear(neurons, neurons)
    initializer(self.dense1.weight)
    self.activation1 = activation
    self.dense2 = nn.Linear(neurons, 1)
    initializer(self.dense2.weight)
    # self.output = nn.Sigmoid() 

  def forward(self, X):
    X = self.dense0(X)
    X = self.activation0(X)
    X = self.dense1(X)
    X = self.activation1(X)
    X = self.dense2(X)
    # X = self.output(X) 
    return X

## Etapa 4: Skorch

In [11]:
classificador_sklearn = NeuralNetBinaryClassifier(module=classificador_torch,
                                                  lr = 0.001,
                                                  optimizer__weight_decay = 0.0001,
                                                  train_split=False)

## Etapa 5: Tuning dos parâmetros

In [12]:
params = {'batch_size': [10],
          'max_epochs': [100],
          'optimizer': [torch.optim.Adam, torch.optim.SGD],
          'criterion': [torch.nn.BCEWithLogitsLoss], #, torch.nn.HingeEmbeddingLoss], 
          'module__activation': [F.relu, F.tanh],
          'module__neurons': [8, 16], 
          'module__initializer': [torch.nn.init.uniform]} # _, torch.nn.init.normal_]}

In [13]:
params

{'batch_size': [10],
 'max_epochs': [100],
 'optimizer': [torch.optim.adam.Adam, torch.optim.sgd.SGD],
 'criterion': [torch.nn.modules.loss.BCEWithLogitsLoss],
 'module__activation': [<function torch.nn.functional.relu(input: torch.Tensor, inplace: bool = False) -> torch.Tensor>,
  <function torch.nn.functional.tanh(input)>],
 'module__neurons': [8, 16],
 'module__initializer': [<function torch.nn.init._make_deprecate.<locals>.deprecated_init(*args, **kwargs)>]}

In [14]:
grid_search = GridSearchCV(estimator=classificador_sklearn, param_grid=params,
                           scoring = 'accuracy', cv = 2)
grid_search = grid_search.fit(previsores, classe)

<ipython-input-10-d2a428839dfb>:6: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  initializer(self.dense0.weight)
<ipython-input-10-d2a428839dfb>:9: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  initializer(self.dense1.weight)
<ipython-input-10-d2a428839dfb>:12: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  initializer(self.dense2.weight)


  epoch    train_loss     dur
-------  ------------  ------
      1    27299.8736  0.7114
      2    24343.2036  0.0356
      3    21607.9807  0.0371
      4    19135.1212  0.0366
      5    16931.4985  0.0383
      6    14986.2365  0.0369
      7    13263.7471  0.0351
      8    11731.9747  0.0399
      9    10367.9509  0.0370
     10     9150.4847  0.0376
     11     8060.7277  0.0377
     12     7080.4200  0.0519
     13     6192.2920  0.0367
     14     5382.8836  0.0391
     15     4640.1404  0.0353
     16     3952.5611  0.0351
     17     3309.9055  0.0368
     18     2702.8676  0.0357
     19     2122.0805  0.0337
     20     1558.6191  0.0359
     21     1003.4047  0.0351
     22      478.4396  0.0396
     23      138.4233  0.0369
     24       90.7293  0.0378
     25       88.3919  0.0370
     26       77.7053  0.0350
     27       70.4368  0.0356
     28       64.3089  0.0359
     29       58.6982  0.0375
     30       54.5391  0.0378
     31       52.0841  0.0380
     32   

<ipython-input-10-d2a428839dfb>:6: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  initializer(self.dense0.weight)
<ipython-input-10-d2a428839dfb>:9: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  initializer(self.dense1.weight)
<ipython-input-10-d2a428839dfb>:12: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  initializer(self.dense2.weight)


      6    11149.3161  0.0396
      7     9765.4888  0.0371
      8     8523.4363  0.0403
      9     7408.3096  0.0383
     10     6405.5098  0.0372
     11     5502.9312  0.0362
     12     4687.3060  0.0383
     13     3944.7730  0.0373
     14     3263.5492  0.0356
     15     2631.7705  0.0367
     16     2037.1366  0.0378
     17     1466.6371  0.0490
     18      907.2859  0.0431
     19      368.1897  0.0426
     20       70.2290  0.0427
     21       64.8931  0.0444
     22       41.8883  0.0487
     23       38.2925  0.0461
     24       34.4358  0.0405
     25       30.3832  0.0363
     26       28.4991  0.0365
     27       26.7977  0.0374
     28       27.0994  0.0429
     29       26.0353  0.0441
     30       24.2245  0.0390
     31       23.3312  0.0366
     32       23.0973  0.0369
     33       22.5641  0.0455
     34       22.1781  0.0395
     35       21.5842  0.0374
     36       20.8129  0.0406
     37       18.3122  0.0392
     38       17.9072  0.0352
     39   

<ipython-input-10-d2a428839dfb>:6: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  initializer(self.dense0.weight)
<ipython-input-10-d2a428839dfb>:9: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  initializer(self.dense1.weight)
<ipython-input-10-d2a428839dfb>:12: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  initializer(self.dense2.weight)


...

      7        2.6482  0.0315
      8        2.5785  0.0305
      9        2.5088  0.0328
     10        2.4391  0.0316
     11        2.3696  0.0277
     12        2.3002  0.0269
     13        2.2310  0.0310
     14        2.1619  0.0369
     15        2.0931  0.0300
     16        2.0245  0.0306
     17        1.9562  0.0305
     18        1.8883  0.0292
     19        1.8208  0.0275
     20        1.7538  0.0302
     21        1.6875  0.0292
     22        1.6219  0.0303
     23        1.5571  0.0289
     24        1.4934  0.0288
     25        1.4308  0.0273
     26        1.3697  0.0314
     27        1.3101  0.0271
     28        1.2523  0.0344
     29        1.1966  0.0300
     30        1.1432  0.0286
     31        1.0923  0.0291
     32        1.0442  0.0273
     33        0.9992  0.0305
     34        0.9573  0.0286
     35        0.9187  0.0290
     36        0.8836  0.0346
     37        0.8518  0.0309
     38        0.8235  0.0303
     39        0.7984  0.0332
     40   

<ipython-input-10-d2a428839dfb>:6: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  initializer(self.dense0.weight)
<ipython-input-10-d2a428839dfb>:9: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  initializer(self.dense1.weight)
<ipython-input-10-d2a428839dfb>:12: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  initializer(self.dense2.weight)
/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:1956: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


      7        3.7531  0.0308
      8        3.6861  0.0320
      9        3.6191  0.0426
     10        3.5520  0.0290
     11        3.4850  0.0276
     12        3.4180  0.0317
     13        3.3511  0.0312
     14        3.2841  0.0272
     15        3.2171  0.0286
     16        3.1501  0.0301
     17        3.0832  0.0296
     18        3.0163  0.0291
     19        2.9494  0.0276
     20        2.8825  0.0317
     21        2.8156  0.0277
     22        2.7488  0.0312
     23        2.6820  0.0287
     24        2.6153  0.0295
     25        2.5486  0.0294
     26        2.4820  0.0293
     27        2.4155  0.0278
     28        2.3491  0.0301
     29        2.2828  0.0284
     30        2.2167  0.0297
     31        2.1507  0.0290
     32        2.0849  0.0290
     33        2.0194  0.0318
     34        1.9541  0.0274
     35        1.8892  0.0313
     36        1.8247  0.0273
     37        1.7607  0.0294
     38        1.6972  0.0278
     39        1.6344  0.0298
     40   

<ipython-input-10-d2a428839dfb>:6: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  initializer(self.dense0.weight)
<ipython-input-10-d2a428839dfb>:9: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  initializer(self.dense1.weight)
<ipython-input-10-d2a428839dfb>:12: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  initializer(self.dense2.weight)


      2    67578.9301  0.1254
      3    51267.8874  0.1034
      4    38467.3813  0.0987
      5    28422.5022  0.1020
      6    20474.0240  0.0978
      7    14073.4598  0.0994
      8     8759.0278  0.0991
      9     4118.9133  0.1014
     10      574.8676  0.0999
     11      151.9111  0.1043
     12      122.1277  0.1131
     13      103.8000  0.1046
     14       87.6229  0.1005
     15       72.9789  0.0991
     16       64.6352  0.1101
     17       63.1611  0.1017
     18       58.8906  0.0996
     19       57.9270  0.1050
     20       53.4348  0.1021
     21       51.8312  0.1105
     22       46.2688  0.1138
     23       45.3725  0.1095
     24       43.1940  0.1065
     25       47.5450  0.0988
     26       54.5306  0.0993
     27       52.1836  0.0994
     28       45.6434  0.1068
     29       41.6177  0.1081
     30       40.6210  0.1241
     31       48.2035  0.1080
     32       52.3442  0.1086
     33       48.4407  0.1063
     34       47.7643  0.0813
     35   

In [15]:
melhores_parametros = grid_search.best_params_
melhor_precisao = grid_search.best_score_

In [16]:
melhores_parametros

{'batch_size': 10,
 'criterion': torch.nn.modules.loss.BCEWithLogitsLoss,
 'max_epochs': 100,
 'module__activation': <function torch.nn.functional.relu(input: torch.Tensor, inplace: bool = False) -> torch.Tensor>,
 'module__initializer': <function torch.nn.init._make_deprecate.<locals>.deprecated_init(*args, **kwargs)>,
 'module__neurons': 16,
 'optimizer': torch.optim.adam.Adam}

In [17]:
melhor_precisao

0.8295218680504077